# imklib

imklib is an experimental library to explore the use of Kotlin extension functions and operator overloading for a NumPy like user experience for the [ImgLib2](https://github.com/imglib/imglib2) Java multi-dimensional image processing library.

## Kotlin Operator Overloading

Operators in Kotlin can be [overloaded](https://kotlinlang.org/docs/reference/operator-overloading.html) for arbitrary types. Certain functions can be marked with the `operator` keyword to overload associated operators, for example to overload the [`+` binary operator](https://kotlinlang.org/docs/reference/operator-overloading.html#arithmetic):
```kotlin
operator fun plus(increment: T): U
```


In [1]:
class Apple
class Orange
data class FruitBowl(var apples: Int = 0, var oranges: Int = 0) {

    private val someNumber = 1

    operator fun plusAssign(apple: Apple) {
        apples += 1
    }
    operator fun plusAssign(orange: Orange) {
        oranges += 1
    }
}

null

In [2]:
val bowl = FruitBowl(oranges=10)
println(bowl)
bowl += Apple()
bowl += Apple()
bowl += Orange()
bowl

FruitBowl(apples=0, oranges=10)


FruitBowl(apples=2, oranges=11)

## Kotlin Extension Functions

Operator overloading is cool but I cannot change upstream classes!! Also, maybe upstream classes are implemented in Java!! Is operator overloading useless? No! Extension functions can be used to add arbitrary methods to existing classes without modifying upstream code:

In [3]:
operator fun FruitBowl.minusAssign(apple: Apple) {
    apples -= 1
}

operator fun FruitBowl.minusAssign(orange: Orange) {
    oranges -= 1
}

bowl -= Orange()
bowl

FruitBowl(apples=2, oranges=10)

### Caveats
 - Internally, extension functions are static methods that take the class object as first argument. Private members or methods cannot be accessed!

In [4]:
fun FruitBowl.someExtension() = this.someNumber

error:  cannot access 'someNumber'

 - Member functions always take precedence over extension functions: Member functions cannot be overridden/replaced by extension functions

In [5]:
operator fun FruitBowl.plusAssign(apple: Apple) {
    oranges += 1
}
val bowl = FruitBowl()
bowl += Apple()
println("With extension function defined in this cell, would expect ${FruitBowl(oranges=1)}")
bowl

With extension function defined in this cell, would expect FruitBowl(apples=0, oranges=1)


FruitBowl(apples=1, oranges=0)

--------------------

### Use Kotlin operator overloading and extension functions for NumPy style tensor operations in ImgLib2!!!!
 - experimental [imklib](https://github.com/hanslovsky/imklib)
 - imklib [examples](https://github.com/hanslovsky/imklib/tree/master/src/test/kotlin/net/imglib2/imklib/examples)
 - also cool with [kscript](https://github.com/holgerbrandl/kscript), [examples](https://github.com/hanslovsky/imklib/tree/026cd65946defc21b9b53161ea348691f3b4f22a/examples/kscript)

 - Add scijava and local maven repositories and maven dependencies

In [6]:
%classpath config resolver maven.scijava.org https://maven.scijava.org/content/groups/public
%classpath config resolver mvnLocal

Added new repo: maven.scijava.org
Added new repo: mvnLocal


In [7]:
%%classpath add mvn
net.imglib2 imglib2 5.7.0
net.imglib2 imklib 0.1.1-SNAPSHOT

 - Import all extensions that are available in `imklib`:

In [8]:
import net.imglib2.imklib.extensions.*

null

 - Some arithmetic operations available for `RealType`, e.g:

In [9]:
import net.imglib2.type.numeric.RealType
import net.imglib2.type.numeric.real.DoubleType

fun <T: RealType<T>> add(dt1: T) {
    val dt2 = dt1 + dt1
    val dt3 = dt1 + dt2
    val dt4 = dt3 + 1.0
//    val dt5 = 123 + dt4
    println("Added:      $dt1 $dt2 $dt3 $dt4")// $dt5")
}

fun <T: RealType<T>> subtract(dt1: T) {
    val dt2 = dt1 - 2.0
    val dt3 = dt1 - dt2
    val dt4 = dt3 - 1.0
//    val dt5 = 123 - dt4
    println("Subtracted: $dt1 $dt2 $dt3 $dt4")// $dt5")
}

fun <T: RealType<T>> multiply(dt1: T) {
    val dt2 = dt1 * dt1
    val dt3 = dt1 * dt2
    val dt4 = dt3 * 2.0
//    val dt5 = 123 * dt4
    println("Multiplied: $dt1 $dt2 $dt3 $dt4")// $dt5")
}

fun <T: RealType<T>> divide(dt1: T) {
    val dt2 = dt1 / 3.0
    val dt3 = dt1 / dt2
    println("Divided:    $dt1 $dt2 $dt3")// $dt4 $dt5")
}

null

In [10]:
add(DoubleType(1.0))
subtract(DoubleType(2.0))
multiply(DoubleType(.133153))
divide(DoubleType(137.0))

Added:      1.0 2.0 3.0 4.0
Subtracted: 2.0 0.0 2.0 1.0
Multiplied: 0.133153 0.017729721408999997 0.0023607655947725766 0.004721531189545153
Divided:    3.0 45.666666666666664 kotlin.Unit


null

Caveat: What is unexpected in the last output line? Name clash between `net.imglib2.type.operators.Div.div` and Kotlin [binary division operator `div`](https://kotlinlang.org/docs/reference/operator-overloading.html#arithmetic)

In [11]:
import net.imglib2.RandomAccessibleInterval
import net.imglib2.imklib.extensions.*
import net.imglib2.img.array.ArrayImgs
import net.imglib2.type.numeric.real.DoubleType
import net.imglib2.util.Intervals

null

 - Access image voxels directly! Warning: This is a convenience method and should be used only for sparse voxel access.

In [12]:
val img1 = ArrayImgs.doubles(
            doubleArrayOf(
                    1.0,  2.0,  3.0,  4.0,
                    5.0,  6.0,  7.0,  8.0,
                    9.0, 10.0, 11.0, 12.0),
            4, 3) as RandomAccessibleInterval<DoubleType>
println("${img1.iterable().map { it.realDouble }}")
println(img1[0, 1])
println(img1[1, 0])
img1

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0]
5.0
2.0


ArrayImg [4x3]

 - Arithmetic operations between images and scalars:

In [13]:
val img2 = img1 + 1.0
println("${img2.iterable().map { it.realDouble }}")
val img2_1 = img1 * 3.0
println("${img2_1.iterable().map { it.realDouble }}")

[2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0]
[3.0, 6.0, 9.0, 12.0, 15.0, 18.0, 21.0, 24.0, 27.0, 30.0, 33.0, 36.0]


null

 - Slicing: Rows

In [14]:
(0 until img1.dimension(1)).forEach {
    val img = img1[AX, it]
    println("dim 1 at $it: $img ${img.iterable().map { it.realDouble }}")
}

dim 1 at 0: IntervalView [(0) -- (3) = 4] [1.0, 2.0, 3.0, 4.0]
dim 1 at 1: IntervalView [(0) -- (3) = 4] [5.0, 6.0, 7.0, 8.0]
dim 1 at 2: IntervalView [(0) -- (3) = 4] [9.0, 10.0, 11.0, 12.0]


null

   - Slicing: Columns

In [15]:
(0 until img1.dimension(0)).forEach {
    val img = img1[AX(it, it), AX]
    println("dim 0 at $it: $img ${img.iterable().map { it.realDouble }}")
}

dim 0 at 0: IntervalView [(0, 0) -- (0, 2) = 1x3] [1.0, 5.0, 9.0]
dim 0 at 1: IntervalView [(1, 0) -- (1, 2) = 1x3] [2.0, 6.0, 10.0]
dim 0 at 2: IntervalView [(2, 0) -- (2, 2) = 1x3] [3.0, 7.0, 11.0]
dim 0 at 3: IntervalView [(3, 0) -- (3, 2) = 1x3] [4.0, 8.0, 12.0]


null

 - sub-sampling

In [16]:
val img3 = img1[AX..2]
println("${Intervals.minAsLongArray(img3).map { it }} ${Intervals.maxAsLongArray(img3).map { it }} ${img3.iterable().map { it.realDouble }}")

[0, 0] [1, 2] [1.0, 3.0, 5.0, 7.0, 9.0, 11.0]


null

 - Element-wise arithmetic operations between images:

In [17]:
// element-wise division
val img4 = img1 / img2
println("${img4.iterable().map { it.realDouble }}")

// elemnt-wise division by scalar followed by exponantiation
val img5 = (img1 / 10.0).exp()
println("${img5.iterable().map { it.realDouble }}")

// add random gaussian noise to each element
val rng = java.util.Random(100)
val img6 = img5.apply({it + 0.1*rng.nextGaussian()}, DoubleType())

[0.5, 0.6666666666666666, 0.75, 0.8, 0.8333333333333334, 0.8571428571428571, 0.875, 0.8888888888888888, 0.9, 0.9090909090909091, 0.9166666666666666, 0.9230769230769231]
[1.1051709180756477, 1.2214027581601699, 1.3498588075760032, 1.4918246976412703, 1.6487212707001282, 1.8221188003905089, 2.0137527074704766, 2.225540928492468, 2.45960311115695, 2.718281828459045, 3.0041660239464334, 3.3201169227365472]


null

 - Cool beakerx/jupyter feature:

In [18]:
val plot = Plot()
plot.setTitle("Exponential")

val line = Line()
line.setY(img5.iterable().map {it.realDouble})
line.setX(img4.iterable().map {it.realDouble})
plot.add(line)

val line2 = Line()
line2.setY(img6.iterable().map {it.realDouble})
line2.setX(img4.iterable().map {it.realDouble})
plot.add(line2)

In [19]:
val plot = Plot()
plot.setTitle("x/(x+1)")

val line = Line()
line.setY(img4.iterable().map {it.realDouble})
line.setX(img1.iterable().map {it.realDouble})
plot.add(line)